### Build an LLM from scratch (i.e. ChatGPT model)

**Chapter 1 - Understanding LLMs**

- LLMs 'understand' human language in that they can generate text that appears coherent/ contextually relevant<br>
<br>

- The 'large' in LLM refers to the large number of parameters and size of the dataset the model is trained on
- LLMs have a 'transformer' architecture, which allows them to pay 'selective' attention to parts of an input
- LLMs fall under the category of deep learning, which is a subset of machine learning; a subset of AI
- LLMs are trained on datasets that allow them to classify things (unlike manual rule-setting)<br>
<br>

- LLMs are best used for tasks that involve parsing and generating text in specialised fields like medicine/ law<br>
<br>

- Most modern LLMs are implemented using PyTorch - domain specific ones can outperform general ones like ChatGPT
- Custom LLMs are also smaller scale and can be deployed from laptops/ phones, where biggers ones are more costly
- Creating an LLM involves 2 phases - pre-training (using large datasets) and fine-tuning (using narrower datasets)
- Pre-training uses raw, unlabelled text, and gives the LLM some simple capabilities like text completion
- Fine-tuning can be 'instruction' or 'classification', which uses 'QnA' style data or 'labelled' data respectively<br>
<br>

- The original transformer architecture was developed to translate English into German and French texts
- Transformers consist of:
    - An 'encoder' that processes the input text into numerical representations
    
    - A 'decoder' that processes the numerical representations and generates output text
- LLMs have a 'self-attention' mechanism that allows the model to weight the importance of different parts of an input
- Generative pre-trained transformers (GPT, like ChatGPT) use this mechanism to perform:
    - Zero shot learning, which are tasks that are carried out without any prior examples
    
    - Few shot learning, which involve some examples the user provides as input<br>
<br>

- Pre-trained models of current ChatGPT versions are versatile and good for being fine-tuned for specific purposes<br>
<br>

- GPT models were pre-trained on a next-word prediction task, which predicts the next word in a text based on the previous words
- This training is a form of self-labeling, where the structure of the data itself is the label (i.e. the predicted word)
- GPT architecture actually only contains the 'decoder' part of the transformer, AKA 'autoregressive' models
    - These models incorporate their previous outputs as inputs for future predictions
- GPT models are also interesting in that they can perform tasks that they were not trained for
    - Language translation is an 'emergent' capability of GPT, showing that diverse tasks do not require diverse models<br>
<br>

- Building an LLM first requires data preparation and implementing the architecture (both can be done low-cost)<br>

**Chapter 2 - Working with Text data**

- Pre-training the LLM involves preparing text data by splitting it into individual word and subword 'tokens'
- These are then encoded into vector representations (i.e. lists containing numbers)<br>
<br>

- Text embedding is the process of converting text into numerical vectors (done so as LLMs cannot process raw text)
- Word embeddings can have more than 1 dimension (i.e. more than 1 number in a list), more dimensions = more computation<br>
<br>

- tokenising is the process of splitting text into tokens, where each word or punctuation is a single tokens<br>
<br>

The following code reads in a short story, "The Verdict", to be used as text data for the tokenisation process

In [47]:
file_path = "the-verdict.txt"

with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print("\nFirst 100 characters: " + str(raw_text[:99]))

Total number of characters: 20479

First 100 characters: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


1. Split the text into singular words and punctuations (i.e. singular tokens)

- Python has a regular expression library that can be used to split text into singular words, as seen in the code below

In [46]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print("Total number of tokens: " + str(len(preprocessed)))
print("\nFirst 10 individual words: " + str(preprocessed[:10]))


Total number of tokens: 4690

First 10 individual words: ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']


2. Convert the tokens (retrieved words and punctuations) into token IDs

- The tokens are stored in an alphabetical vocabulary, where each token has a unique ID

In [45]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print("Total vocab size: " + str(vocab_size))

vocab = {token:integer for integer, token in enumerate(all_words)}

print("\nTokens 21 - 25 of the vocab:")
for i, item in enumerate(vocab.items()):
    if i > 20 and i <= 25:
        print(item)

Total vocab size: 1130

Tokens 21 - 25 of the vocab:
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)


3. Apply the vocabulary to convert new text data into tokens

- This models the encode and decode processes of a transformer, which can be carried out by a tokeniser class

- However, this will only be able to tokenise text that is within the vocabulary (duh)

In [44]:
class Tokeniser:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\\])', r'\1', text)

        return text
    
tokeniser = Tokeniser(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokeniser.encode(text)

print("Token ids of the words in the text: " + str(ids))
print("\nDecoded result of the tokenised words: " + str(tokeniser.decode(ids)))

Token ids of the words in the text: [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]

Decoded result of the tokenised words: " It ' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


4. Modify the tokeniser to handle unknown words

- Tokens can be added to the vocabulary to represent unknown words and text separations

- The encode function of the tokeniser class can be modified to tokenise these special cases in the text data

In [43]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print("New amount of tokens in vocab: " + str(len(vocab.items())))

def new_encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int
                    else "<|unk|>" for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

Tokeniser.encode = new_encode

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

print("\nJoined sample text with separation: " + text)

new_tokeniser = Tokeniser(vocab)

print("\nDecoded encoded text: " + new_tokeniser.decode(new_tokeniser.encode(text)))

New amount of tokens in vocab: 1132

Joined sample text with separation: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.

Decoded encoded text: <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


*Unknown words can also be handled by tokenising them through Byte Pair Encoding (BPE)*

- A BPE tokeniser breaks down unknown words into subwords which exist within the vocab and have token ids

- On subsequent iterations, the tokeniser merges frequent characters into larger words, which increase the vocab

*This was used to train models like GPT-2 and GPT-3, the original models for ChatGPT

In [42]:
from importlib.metadata import version
import tiktoken

print("Installed tiktoken version: ", version("tiktoken"))

tokeniser = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
ids = tokeniser.encode(text, allowed_special={"<|endoftext|>"})

print("\nTiktoken tokeniser encoded ids: " + str(ids))

words = tokeniser.decode(ids)

print("\nTiktoken tokeniser decoded words: " + words)

Installed tiktoken version:  0.12.0

Tiktoken tokeniser encoded ids: [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]

Tiktoken tokeniser decoded words: Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


5. Generate input-target pairs from the data using a 'sliding window'

- LLMs are pretrained by predicting the next word in a text, where the predicted words are taken in as input each time

- The 'sliding window' takes a group of words in a text for each prediction (i.e. a single context), and moves across it to continue predicting

In [41]:
the_verdict = raw_text
enc_text = tokeniser.encode(the_verdict)

print("Number of tokens in the verdict, as tokenised by the BPE tokeniser: " + str(len(enc_text)))

enc_sample = enc_text[50:]

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1: context_size + 1]

print("\nIllustration of how the sliding window works:")
print(f"x:  {x}")
print(f"y:       {y}\n")

for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample [i]
    print(tokeniser.decode(context), " --->", tokeniser.decode([desired]))

Number of tokens in the verdict, as tokenised by the BPE tokeniser: 5145

Illustration of how the sliding window works:
x:  [290, 4920, 2241, 287]
y:       [4920, 2241, 287, 257]

 and  --->  established
 and established  --->  himself
 and established himself  --->  in
 and established himself in  --->  a


6. Implement a more efficient way of iterating over the text data and returning the input-target pairs

- PyTorch is a library that can return data as 'tensors' (i.e. multidimensional arrays)

- The input tensor contains the text data the LLM sees and the target tensor contains the targets the LLM predicts

In [39]:
import torch

from torch.utils.data import Dataset, DataLoader

# creating a class to initialise a text dataset
class GPTDataset(Dataset):
    def __init__(self, txt, tokeniser, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokeniser.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
    
# creating a function to create a dataloader
def create_dataloader(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokeniser = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokeniser, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

dataloader = create_dataloader(the_verdict, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)

print("First input and target of the LLM: " + str(first_batch))

second_batch = next(data_iter)

print("\nSecond input and target of the LLM: " + str(second_batch))

First input and target of the LLM: [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

Second input and target of the LLM: [tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


7. Convert the token IDs into embeddings (i.e. random values that give a weight to each token)

- Each token in the vocab is assigned a number of random output dimensions (3 for this example)

- When any token is read in as input, these output dimensions will be looked up in the vocab (i.e. embedding layer) and returned

In [38]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

# pytorch adds random values to each of the output dimensions of each token
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print("Embeddings of tokens in vocab:\n", embedding_layer.weight)

print("\nEmbeddings for token 3/ row 4 in layer:\n", embedding_layer(torch.tensor([3])))

print("\nEmbeddings for tokens in input_ids:\n", embedding_layer(input_ids))


Embeddings of tokens in vocab:
 Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)

Embeddings for token 3/ row 4 in layer:
 tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)

Embeddings for tokens in input_ids:
 tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


8. Change the token embeddings to account for different positions in a text input

- When a text input has duplicate words, the same token embeddings are returned for those 2 words (this is wrong)

- To avoid this, the original token embeddings can be modified to indicate their positions

In [86]:
# using the size and dimensions of the BPE tokeniser as an example
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# using 'the-verdict' text as the text input for the dataloader
max_length = 4
dataloader = create_dataloader(the_verdict, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

# each 'batch' corresponds to the number of rows and the 'max_length' is the number of tokens in each row
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

# this adds 256 random values to each of the token ids (the gpt-3 model added 12,288 :O)
token_embeddings = token_embedding_layer(inputs)

print("\nToken embeddings shape:\n", token_embeddings.shape)

# to modify the embeddings to indicate their position, a new layer must be created
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))

print("\nPosition embeddings shape:\n", pos_embeddings.shape)

# add the 2 layers together to create the final input embeddings for the LLM
input_embeddings = token_embeddings + pos_embeddings

print("\nFinal input embeddings shape:\n", input_embeddings.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])

Token embeddings shape:
 torch.Size([8, 4, 256])

Position embeddings shape:
 torch.Size([4, 256])

Final input embeddings shape:
 torch.Size([8, 4, 256])


The embeddings can be thought of as layers, where the:

- First layer: contains multiple input batches

- Second layer: each input batch contains multiple tokens

- Third layer: each token contains multiple embeddings (256 in the code above)

**The initial text data is now prepared for processing by the main LLM modules!**